In [18]:
import pandas as pd
import numpy as np
from covid import utils

paths = utils.create_named_tuple('filepaths.txt')
config = utils.create_named_tuple(paths.config)
age_labels = utils.generate_labels_from_bins(config.age_bins)
population = pd.read_csv('data/temp_pop.csv')

In [18]:
population

,region_id,region,0-6,7-15,16-20,21-40,41-66,67+,population
0,301,OSLO,55837,64036,33296,257985,207877,77749,696780
1,1101,EIGERSUND,1132,1719,955,3642,4898,2439,14785
2,1103,STAVANGER,11735,16218,8511,41917,47283,18451,144115
3,1106,HAUGESUND,2636,3996,2314,9836,12823,5707,37312
4,1108,SANDNES,7589,10110,5042,22845,25653,9206,80445
...,...,...,...,...,...,...,...,...,...
351,5440,BERLEVÅG,47,61,46,219,341,214,928
352,5441,TANA,166,232,154,624,1052,600,2828
353,5442,NESSEBY,33,86,40,197,295,229,880
354,5443,BÅTSFJORD,150,197,124,622,761,346,2200


In [51]:
df = pd.read_csv('data/Sysselsatte.csv')
df['age_low'] = df.alder.apply(lambda x: str(x).split("-")[0]).astype(int)
df['age_high'] = df.alder.apply(lambda x: str(x).split("-")[1][:2]).astype(int)
df.drop(columns=["alder"], inplace=True)
df.rename({"Sysselsatte personer": "employed"}, axis=1, inplace=True)

In [82]:
percent_commuters = 0.36
bins = config.age_bins
df = pd.read_csv('data/employed_per_age.csv')
df.age = pd.cut(df['age'], bins=bins+[110], labels=age_labels, include_lowest=True)
commuters = df.groupby('age').sum()['employed'].to_numpy() * percent_commuters
sum_age_groups = population[population.columns[2:-1]].sum().to_numpy()
age_group_commuter_percent = commuters/sum_age_groups
age_group_flow_scaling = age_group_commuter_percent/age_group_commuter_percent.sum()

In [7]:
df = pd.read_csv('data/death_by_age.csv')
df

,age_low,age_high,infected,deaths
0,0,9,8929,0
1,10,19,16651,0
2,20,29,21640,0
3,30,39,18437,0
4,40,49,16991,10
5,50,59,14011,27
6,60,69,6366,76
7,70,79,3165,155
8,80,89,1426,248
9,90,105,494,187


In [8]:
age = []
infected = []
dead = []
for row in df.iterrows():
    info = row[1]
    age_diff = info.age_high - info.age_low
    for i in range(info.age_low, info.age_high+1):
        age.append(i)
        infected.append(info.infected/age_diff)
        dead.append(info.deaths/age_diff)

In [15]:
df_new = pd.read_csv('data/death_by_age.csv')
df_new

,age,infected,dead
0,0,992,0
1,1,992,0
2,2,992,0
3,3,992,0
4,4,992,0
...,...,...,...
101,101,32,12
102,102,32,12
103,103,32,12
104,104,32,12


In [21]:
bins = config.age_bins
labels = age_labels
df = pd.read_csv('data/death_by_age.csv')
df.age = pd.cut(df['age'], bins=bins+[110], labels=labels, include_lowest=True)
infected = df.groupby('age').sum()['infected'].to_numpy()
dead = df.groupby('age').sum()['dead'].to_numpy()
dead/infected

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.27257232e-05,
       2.53387389e-03, 8.49410851e-02])

In [23]:
dead

array([  0,   0,   0,   1,  95, 656], dtype=int64)

In [22]:
infected

array([ 6944, 14076,  9804, 44003, 37492,  7723], dtype=int64)

In [25]:
np.round(dead/infected, 3)

array([0.   , 0.   , 0.   , 0.   , 0.003, 0.085])